In [4]:
import base64
import json
import requests
from PIL import Image
import io

def display_image(image_path):
    """Optional: show image with PIL"""
    image = Image.open(image_path)
    image.show()

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def extract_passport_info_with_ollama(image_path):
    display_image(image_path)
    base64_image = encode_image_to_base64(image_path)

    prompt = (
        "Analyze the following image of a Polish passport and return a JSON object with:\n"
        "- `passport_number`: string\n"
        "- `full_name`: string\n"
        "- `expiry_date`: string in YYYY-MM-DD format\n"
        "- `confidence`: number from 0 to 1 based on how sure you are\n\n"
        "Return only the JSON.\n"
        "<image>\n"
    )

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llava",
            "prompt": prompt,
            "images": [base64_image],
            "stream": False
        }
    )

    if response.status_code != 200:
        raise Exception(f"Ollama error: {response.text}")

    try:
        content = response.json()["response"]
        json_str = content.strip().split("```json")[-1].split("```")[0] if "```json" in content else content
        result = json.loads(json_str)
    except Exception as e:
        print("Failed to parse JSON:", e)
        print("Raw response:", content)
        return None

    return result

data = extract_passport_info_with_ollama("/Users/gro/Desktop/passport_test.jpg")
if data:
    print("\n🧾 Dane z paszportu:")
    print(json.dumps(data, indent=4, ensure_ascii=False))


🧾 Dane z paszportu:
{
    "passport_number": "A09",
    "full_name": "Jan Kowalski",
    "expiry_date": "2024-12-31",
    "confidence": 0.75
}
